In [402]:
import numpy as np

Data

In [403]:
X = np.array([7, 4])
target = np.array([0,1])
neurons = 3
neurons_input = np.shape(X)[0]
neurons_output = np.shape(target)[0]

print 
weights_hidden = np.random.rand(neurons,neurons_input)                    
weights_outputs = np.random.rand(neurons_output,neurons)
learning_rate = 5
iterations = 4

Activation function (sigmoid)

In [404]:
def sigmoid(X):
    return 1 / (1 + np.exp(-X))

softMax function to normalize the output

In [405]:
def softmax(vec):
  exponential = np.exp(vec)
  probabilities = exponential / np.sum(exponential)
  return probabilities


calculating output at each node from net and sigmoid

In [406]:
def calc_Net (input_vector, weight_vector):
    return np.dot(input_vector,weight_vector)


In [407]:
def calc_output (input_vector, weight_vector):
    return sigmoid(calc_Net(input_vector, weight_vector))

calculate output at hidden layer

In [408]:
def HiddenLayerOutput_calc (input_vector,weights_hidden,N):
    HiddenLayerOutput = []
    for i in range (0,N):
        HiddenLayerOutput.append(calc_output(input_vector,weights_hidden[i]))
    HiddenLayerOutput = np.array(HiddenLayerOutput)
    return HiddenLayerOutput


calculate output at putput layer

In [409]:
def OutputLayerOutput_calc (HiddenLayerOutput,weights_outputs):
    OutputLayerOutput = []
    for i in range (0,2):
        OutputLayerOutput.append(calc_output(HiddenLayerOutput,weights_outputs[i]))
    OutputLayerOutput = np.array(OutputLayerOutput)
    OutputLayerOutput = softmax(OutputLayerOutput)
    return OutputLayerOutput



Walking forward to calculate outputs

In [410]:
def forward (input_vector, weights_hidden, weights_outputs,N):
    weights_hidden = np.array(weights_hidden)
    weights_outputs = np.array(weights_outputs)       

    
    HiddenLayerOutput = HiddenLayerOutput_calc(input_vector,weights_hidden,N)

    OutputLayerOutput = OutputLayerOutput_calc(HiddenLayerOutput, weights_outputs)

    

    return OutputLayerOutput
    

Calculating Error

In [411]:
def calc_Error (Y, target):
    sum = 0.0
    for i in range (0,target.size):
        sum += (Y[i]-target[i])**2
    return sum

Walking backward to update weight

At the output layer

Calculate segma at output layer

In [412]:
def calc_segma_output (target,Y):
    output_segma = []
    for i in range (0, target.size): 
        output_segma.append((target[i]-Y[i]) * Y[i] * (1 - Y[i]))
    output_segma = np.array(output_segma)
    return output_segma


from segma calculate delta

In [413]:
def calc_delta_w (learning_rate,segma,output,delta ):
    r,c = delta.shape
    for i in range (0,r):
        for j in range (0, c):
            delta[i][j] = learning_rate * segma[i] * output[j]
    return delta




from delta update the weight

In [414]:
def Updated_weight_fun (old_W,delta,updated):
    updated = old_W + delta
    return updated


At hidden layer

calculate segma at hidden layer and calculate delta and updated hidden weights 

In [415]:
def calc_segma_hidden (output_segma,Updated_weight,prev_output):
    hidden_segma = []
    sum = 0.0
    rows, columns = Updated_weight.shape
    for i in range (0,columns):       
        for j in range (0, rows):           
            sum += output_segma[j]*Updated_weight[j][i]    
        segma = sum * prev_output[i]*(1- prev_output[i])
        hidden_segma.append(segma)
    return hidden_segma

walk backward to update weights and forward again to calcilate outputs with the updated weights

In [416]:
def backward (target,Y,X,weights_hidden,weights_outputs,learning_rate,N):
    output_segma = calc_segma_output (target,Y)
    prev_output = HiddenLayerOutput_calc(X,weights_hidden,N)

    delta_w = np.ones(((output_segma.size,prev_output.size)))
    delta_w = calc_delta_w (learning_rate,output_segma,prev_output,delta_w)

    Updated_weight = np.ones(((output_segma.size,prev_output.size)))
    Updated_weight = Updated_weight_fun (weights_outputs,delta_w,Updated_weight)

    hidden_segma = calc_segma_hidden (output_segma,Updated_weight,prev_output)
    hidden_segma = np.array(hidden_segma)

    delta_w_hidden = np.ones(((hidden_segma.size,X.size)))
    delta_w_hidden = calc_delta_w (learning_rate,hidden_segma,X,delta_w_hidden)

    Updated_weight_hidden = np.ones(((hidden_segma.size,X.size)))
    Updated_weight_hidden = Updated_weight_fun(weights_hidden,delta_w_hidden,Updated_weight_hidden)

    Y_new = forward (X,Updated_weight_hidden , Updated_weight,N)
    return Y_new, Updated_weight_hidden,Updated_weight



BACKPRPAGATION approach

In [417]:
Y = forward (X, weights_hidden, weights_outputs,neurons)
Error_1 = calc_Error (Y, target)
print ("first prediction", Y)
print ("first error : ",Error_1)
print ("--------------------------------------")

def backProP (target,Y,X,weights_hidden,weights_outputs,learning_rate,iterations,N):
    y_new,Wh_new,Wo_new = backward (target,Y,X,weights_hidden,weights_outputs,learning_rate,N)
    Error = calc_Error (y_new, target)
    print ("second  prediction", y_new)
    print("error in the second iteration :", Error)
    print ("--------------------------------------")

    for i in range (0,iterations-2):
        y_new,Wh_new,Wo_new = backward (target,Y,X,Wh_new,Wo_new,learning_rate,N)
        Error = calc_Error (y_new, target)
        print (i+3,"th  prediction", y_new)
        print("error in the",i+3,"th iteration = ", Error)
        print ("--------------------------------------")

backProP (target,Y,X,weights_hidden,weights_outputs,learning_rate,iterations,neurons)


first prediction [0.46107749 0.53892251]
first error :  0.4251849001249325
--------------------------------------
second  prediction [0.3224064 0.6775936]
error in the second iteration : 0.20789177338173326
--------------------------------------
3 th  prediction [0.2795905 0.7204095]
error in the 3 th iteration =  0.15634169137229958
--------------------------------------
4 th  prediction [0.27089238 0.72910762]
error in the 4 th iteration =  0.1467653657826077
--------------------------------------
